# fSEND fMRI Run Sequences

In this notebook, I generate .JSON files that will determine how the experimental task will prsent the stimuli during the scan. The paradigm includes 32 videos from the <a href="https://arxiv.org/abs/1912.05008">Standford Emotional Narratives Dataset</a>, as well as two short, additional videos which involve internal narratives from the main characters of each video. 

The goal of this code is to shuffle all 5 runs such that their overall order varies per participant, but the internal sequence of the individual orders are preserved; that is, the 8 videos inside of a given run should always be shown in the same sequence each time. 

Additionally, the internal narrative films should also be shuffled, but not intermixed with the SEND videos; participants with even subject ID's will see the narrative films first, and the SEND videos second. 

In [1]:
import random
import pandas as pd
import json

In [2]:
out_path = '/Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/stim-shuffled/'

In [3]:
# Four SEND runs, 8 videos per run [do not shuffle]
send_run1 = ['ID120_vid4','ID116_vid2','ID147_vid4','ID112_vid1','ID180_vid6','ID156_vid3','ID130_vid6','ID164_vid3']
send_run2 = ['ID131_vid2','ID118_vid1','ID128_vid5','ID117_vid4','ID170_vid7','ID137_vid6','ID174_vid2','ID181_vid6']
send_run3 = ['ID145_vid4','ID153_vid3','ID165_vid7','ID113_vid4','ID179_vid3','ID169_vid2','ID141_vid1','ID171_vid5']
send_run4 = ['ID111_vid3','ID124_vid6','ID161_vid1','ID168_vid1','ID121_vid6','ID123_vid3','ID127_vid3','ID129_vid6']

send_runs = [send_run1,send_run2,send_run3,send_run4]

# One NarratedThought run, shuffled order
narr_v1 = ['physical_v1','stutter_v1']
narr_v2= ['physical_v2','stutter_v2']

for i in range(1,5):
    random.seed(i)
    random.shuffle(send_runs)
    random.shuffle(narr_v1)
    
    run_sequence = []
    
    if i % 2 == 0: #Even-numbered subjects among the first 25 will watch V1 NarratedThought videos first, SEND second.
        run_sequence.append(narr_v1)
        for sub_list in send_runs:
            run_sequence.append(sub_list)
    else:
        for sub_list in send_runs:
            run_sequence.append(sub_list)
        run_sequence.append(narr_v1)
        
    sequence_dict = {}

    for ix, sub_list in enumerate(run_sequence):
        sequence_dict[str(ix+1)] = {}
        for num,video in enumerate(sub_list):
            sequence_dict[str(ix+1)][sub_list[num]] = {}
            
    with open(out_path+'sub_00'+str(i)+'.json', 'w') as fp:
        json.dump(sequence_dict, fp)
        